<a href="https://colab.research.google.com/github/kraslav4ik/Recruitment-data-analysis-task/blob/main/task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Krasnoperov Viacheslav

BitPeak recruitment task #2

1. Using sqlite3 , create a new database with the FlightLeg table , which will contain information about the flights of the planes, with the following columns:  
  * id - numeric identifier assigned from the sequence  
  * tailNumber - aircraft identifier  
  * sourceAirportCode, destinationAirportCode - three-letter airport code (according to IATA)  
  * sourceCountryCode, destinationCountryCode - three-letter country code (according to ISO 3166-1 Alpha-3)  
  * departureTimeUtc, landingTimeUtc - date and time (accurate to the second), respectively, of departure and landing (in UTC)  

2. Fill in the FlightLeg table with data from the file: https://bitpeak.pl/datasets/flightlegs.csv  

3. Add two new columns to the table:  

  * flightDuration - fill it with values ​​representing flight duration in minutes (rounded to the nearest integer)  

  * flightType - fill it with the flight type values: domestic (value 'D' = domestic) or foreign (value 'I' = international); a domestic flight is a flight that begins and ends in the same country  

4. Implement the logic that answers the following questions:  
  1. Which aircraft made the most flights?  
  2. Which plane flew the most minutes?  
  3. Which flight, broken down into domestic and foreign ones, was the shortest and which was the longest, and how many minutes was it?  
  4. (optional) How many erroneous flight records are there, which indicate that the aircraft was performing more than one flight at the same time? View all pairs of such conflicting flights.  

FOL, Download csv file and create database file in the root directory

In [89]:
!FILE="./flightlegs.csv"; if [ -f "$FILE" ]; then echo "$FILE exists"; else wget "https://bitpeak.pl/datasets/flightlegs.csv"; fi
!touch "task_db.db"

./flightlegs.csv exists


Import all necessary libraries

In [90]:
import sqlite3
import pandas as pd
from collections import defaultdict, Counter
from datetime import datetime

Describe our path's variables, connect to database, "connect", "cursor", paths and answers dict are global variables.

In [91]:
db_path = './task_db.db'
csv_path = './flightlegs.csv'

connect = sqlite3.connect(db_path)
cursor = connect.cursor()
answers = {}

Create table if it isn't exist

In [92]:
def create_table() -> None:
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = set([name[0] for name in cursor])
    if 'FlightLeg' not in tables:
        cursor.execute('''CREATE TABLE FlightLeg
                        (id INTEGER, tail INTEGER, sourceAirportCode TEXT(3),
                        destinationAirportCode TEXT(3), sourceCountryCode TEXT(3), destinationCountryCode TEXT(3),
                        departureTimeUtc TEXT, landingTimeUtc TEXT);''')
        connect.commit()


In the function "csv_data_into_db", read the csv file into pandas DataFrame (assign the columns's names the same as in SQL table). Then, check that our SQL table doesn't contain any records and load all DataFrame's content into table using Dataframe's "to_sql" method

In [93]:
def csv_data_into_db(csv_path) -> None:
    col_names = ["tail", "sourceAirportCode", "sourceCountryCode", "destinationAirportCode",
                "destinationCountryCode", "departureTimeUtc", "landingTimeUtc"]
    df = pd.read_csv(csv_path, names=col_names, skiprows=[0], sep=';')
    cursor.execute('SELECT * FROM FlightLeg;')
    table_rows = [row for row in cursor]
    if not table_rows:
        df.to_sql('FlightLeg', connect, if_exists='append', index_label='id')
        connect.commit()

In next function, add columns "flightDuration" and "flightType" using SQL ALTER TABLE.

1.   For counting days, convert landing time and departure time into Julianday format, find their difference and convert it into minutes, rounded to integer value
2. To set a flight type for each flight, use "CASE WHEN x THEN y ELSE z END"

In [94]:
def add_columns() -> None:
    cols_names = set([description[0] for description in cursor.description])
    if 'flightDuration' not in cols_names:
        cursor.execute("ALTER TABLE FlightLeg ADD COLUMN flightDuration INTEGER;")
        cursor.execute("""UPDATE FlightLeg
                SET flightDuration = ROUND((JULIANDAY(landingTimeUtc) - JULIANDAY(departureTimeUtc)) * 1440)""")
        connect.commit()
    if 'flightType' not in cols_names:
        cursor.execute("ALTER TABLE FlightLeg ADD COLUMN flightType TEXT(1);")
        cursor.execute("""UPDATE FlightLeg SET flightType = CASE WHEN
        sourceCountryCode = destinationCountryCode THEN 'D' ELSE 'I' END;""")
        connect.commit()

Next step, in func "get_flights_info" we collecting info about flights into python dict. Because, it's quite unconvinient to iterate through all the records using SQL querries, I chose to do it in python

1.   Firstly, select plane number, time duration and dep/arr time for each flight
2.   Iterate through the records and collect data for each plane:
      *   every iteration add 1 flight to get final flight count for question #1
      *   every iteration add minutes from each plane's flight to get the all minutes in the sky for question #2 
      *   every iteration collect the departure and arrival time for question #4

In [95]:
def get_flights_info() -> tuple:
    planes = defaultdict(Counter)
    planes_times = defaultdict(list)

    cursor.execute("SELECT tail, flightDuration, departureTimeUtc, landingTimeUtc FROM FlightLeg;")
    for (tail, flightDuration, departureTimeUtc, landingTimeUtc) in cursor:
        planes[tail]["num_of_flights"] += 1
        planes[tail]["fly_time"] += flightDuration
        planes_times[tail].append({"dep": datetime.strptime(departureTimeUtc, '%Y-%m-%d %H:%M:%S'),
                                       "arr": datetime.strptime(landingTimeUtc, '%Y-%m-%d %H:%M:%S')})

    return planes, planes_times

In next two functions I created lists "flights" and "flight_times" using "sorted" built-in function, and "num_of_flights", "fly_time" as keys, respectively. Then took planes, having these biggest values and add them to the answer(considered the case, where are several planes with most flights/minutes)

In [96]:
def first_question(planes: dict) -> None:
    flights = sorted(planes.items(), key=lambda x: x[1]["num_of_flights"], reverse=True)
    most_flights = []
    for flight in flights:
        if flight[1]["num_of_flights"] != flights[0][1]["num_of_flights"]:
            break
        most_flights.append(flight[0])
    
    answers["1. Which aircraft made the most flights?"] = ', '.join(most_flights)
    
def second_question(planes: dict) -> None:
    flight_times = sorted(planes.items(), key=lambda x: x[1]["fly_time"], reverse=True)
    longest_flights = []
    for flight in flight_times:
        if flight[1]["fly_time"] != flight_times[0][1]["fly_time"]:
            break
        longest_flights.append(flight[0])

    answers["2. Which aircraft flew the most minutes?"] = ', '.join(longest_flights)

For 3th question use SQL querries 4 times to get the shortest/longest domestic and international flights. Add info about these flights to the dict with answers

In [97]:
def third_question():
    cursor.execute("""SELECT tail, departureTimeUtc, flightDuration FROM FlightLeg WHERE flightType = 'D' AND
                        flightDuration = (SELECT MAX(flightDuration) FROM FlightLeg WHERE flightType = 'D');""")
    longest_d = [f"{tail}, {departureTimeUtc}, {flightDuration}"
                for (tail, departureTimeUtc, flightDuration) in cursor]
    cursor.execute("""SELECT tail, departureTimeUtc, flightDuration FROM FlightLeg WHERE flightType = 'D' AND
                        flightDuration = (SELECT MIN(flightDuration) FROM FlightLeg WHERE flightType = 'D');""")
    shortest_d = [f"{tail}, {departureTimeUtc}, {flightDuration}"
                  for (tail, departureTimeUtc, flightDuration) in cursor]
    cursor.execute("""SELECT tail, departureTimeUtc, flightDuration FROM FlightLeg WHERE flightType = 'I' AND
                        flightDuration = (SELECT MAX(flightDuration) FROM FlightLeg WHERE flightType = 'I');""")
    longest_i = [f"{tail}, {departureTimeUtc}, {flightDuration}"
                  for (tail, departureTimeUtc, flightDuration) in cursor]
    cursor.execute("""SELECT tail, departureTimeUtc, flightDuration FROM FlightLeg WHERE flightType = 'I' AND
                        flightDuration = (SELECT MIN(flightDuration) FROM FlightLeg WHERE flightType = 'I');""")
    shortest_i = [f"{tail}, {departureTimeUtc}, {flightDuration}"
                  for (tail, departureTimeUtc, flightDuration) in cursor]
    answers["3. Longest domestic"] = ' | '.join(longest_d)
    answers["3. Shortest_domestic"] = ' | '.join(shortest_d)
    answers["3. Lontest_international"] = ' | '.join(longest_i)
    answers["3. Shortest_international"] = ' | '.join(shortest_i)
    return 

For 4th question, finding overlaping in time periods is the same as finding the overlaping sets. I implemented next logic:

For each aircraft:

1.   Sort list with departure and arrival times(key=Arriving time).  
2.   Iterate through this list, refreshing, if there are no intersections, departure time to compare to the current arrival time. Intersection now is possible if and only if the considering plane's departure time <= minimum from the previous arrival times
3. If there is overlapping, select info about these flights from table and add them to the result. In this case, saving arrival value = min(current arrival, previous arrival)



In [98]:
def fourth_question(planes_times) -> None:
    mistakes = defaultdict(list)
    for plane in planes_times:
        if len(planes_times) == 1:
            continue
        sorted_flights = sorted(planes_times[plane], key=lambda x: x["arr"])
        prev_arr = sorted_flights[0]["arr"]
        prev_dep = sorted_flights[0]["dep"]
        for flight in sorted_flights[1:]:
            if flight["dep"] >= prev_arr:
                prev_arr = flight["arr"]
                prev_dep = flight["dep"]
                continue
            mistakes[plane].append(f'{flight["dep"]} - {flight["arr"]} and {prev_dep} - {prev_arr}')
            prev_arr = min(flight["arr"], prev_arr)
    answers["4. Error records"] = ''.join([f"{plane}: {mistakes[plane]}" for plane in mistakes])

Call each of the functions and print the answer

In [99]:
create_table()
csv_data_into_db(csv_path=csv_path)
add_columns()
flights_info = get_flights_info()[0]
flights_times_info = get_flights_info()[1]
first_question(flights_info)
second_question(flights_info)
third_question()
fourth_question(flights_times_info)
for q, a in sorted(answers.items()):
        print(f'{q}: {a}')

1. Which aircraft made the most flights?: A6-ENE, A6-ENS
2. Which aircraft flew the most minutes?: A6-EVF
3. Longest domestic: A6-EES, 2021-11-30 09:20:00, 126
3. Lontest_international: A6-EWH, 2021-11-29 06:32:00, 966
3. Shortest_domestic: A6-ECH, 2021-11-30 09:32:00, 61
3. Shortest_international: A6-ENE, 2021-11-29 13:35:00, 42 | A6-EPF, 2021-11-27 13:44:28, 42 | A6-EGQ, 2021-11-28 13:35:28, 42
4. Error records: A6-ECS: ['2021-11-30 15:45:00 - 2021-11-30 19:59:00 and 2021-11-30 05:14:44 - 2021-11-30 15:50:00']
